In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import corner

In [61]:
prior_file = '../pbjam/data/prior_data.csv'

prior = pd.read_csv(prior_file)
prior = prior.drop(columns=['ID', 'eps_g', 'p_L0', 'p_D0', 'alpha_g', 'd01', 'u1', 'u2', 'DPi1'])
prior = prior.dropna()
''' If you want to reduce the volume of the prior - here is the place to do it!'''
prior = prior[prior.numax > np.log10(800.0)]

prior.head()

,numax,dnu,teff,bp_rp,eps_p,d02,alpha_p,env_height,env_width,mode_width,H1_nu,H1_exp,H_power,H2_nu,H2_exp
711,2.993137,1.748721,3.743902,0.936217,1.437755,0.601810,-3.268724,0.584318,1.960230,-0.187943,2.970684,3.237452,3.772420,2.425723,3.020671
1163,3.056376,1.790688,3.763128,0.862216,1.382435,0.778154,-3.297644,0.656435,1.980680,-0.134599,2.994326,3.032668,3.492433,2.416618,2.389446
1604,2.932298,1.684309,3.736556,0.970967,1.397454,0.716601,-2.758143,1.042723,2.005202,-0.106269,2.854863,4.272459,3.671218,2.329011,3.100447
1713,2.925378,1.692291,3.772542,0.843357,1.432931,0.707959,-2.534071,0.681062,1.997663,-0.149565,2.874650,3.711477,3.680680,2.349129,2.790310
2171,3.103480,1.833976,3.789440,0.738458,1.413263,0.728377,-2.887189,0.541177,2.225190,0.019924,3.054797,3.057458,3.614931,2.541488,3.140720


In [66]:
class params():
    
    
    def __init__(self, prior: pd.core.frame.DataFrame):
        self.prior_cloud = prior
        self.obs = {}
        
    def set_obs(self, name: str, vals: np.array):
        ''' Set an observable in the observable dictionary
        
        name: str
            name of the observable that matches an observable in the prior
            
        vals: arraylike
            [mean, uncertainty]
        '''
        self.obs[name] = vals
        
    def run_pca(self, n_components=5):
        ''' Run PCA on the prior to get a continuous latent representation '''
        X = self.prior_cloud.values
        from sklearn.decomposition import PCA
        self.pca = PCA(n_components=n_components)
        latent = self.pca.fit_transform(X)
        print(f'PCA Explained variance total = {np.sum(self.pca.explained_variance_ratio_)}')
        self.latent_mean = np.mean(latent, axis=0)
        self.latent_std = np.std(latent, axis=0)
        
    def prior_transform(self, u):
        ''' Transform from unit cube to latent space '''
        return u * self.latent_std + self.latent_mean
    
    def likelihood(self, latent):
        p = self.pca.inverse_transform(latent)
        print(p)
        
        
    def __call__(self):
        print(self.prior_cloud.head())


In [67]:
p = params(prior)
p()
p.set_obs('dnu', [135.0, 1.0])
p.set_obs('numax', [3050.0, 5.0])
p.set_obs('teff', [np.log10(5777.), 70.0])
p.run_pca()

u = np.random.rand(5)
latent = p.prior_transform(u)
p.likelihood(latent)

         numax       dnu      teff     bp_rp     eps_p       d02   alpha_p  \
711   2.993137  1.748721  3.743902  0.936217  1.437755  0.601810 -3.268724   
1163  3.056376  1.790688  3.763128  0.862216  1.382435  0.778154 -3.297644   
1604  2.932298  1.684309  3.736556  0.970967  1.397454  0.716601 -2.758143   
1713  2.925378  1.692291  3.772542  0.843357  1.432931  0.707959 -2.534071   
2171  3.103480  1.833976  3.789440  0.738458  1.413263  0.728377 -2.887189   

      env_height  env_width  mode_width     H1_nu    H1_exp   H_power  \
711     0.584318   1.960230   -0.187943  2.970684  3.237452  3.772420   
1163    0.656435   1.980680   -0.134599  2.994326  3.032668  3.492433   
1604    1.042723   2.005202   -0.106269  2.854863  4.272459  3.671218   
1713    0.681062   1.997663   -0.149565  2.874650  3.711477  3.680680   
2171    0.541177   2.225190    0.019924  3.054797  3.057458  3.614931   

         H2_nu    H2_exp  
711   2.425723  3.020671  
1163  2.416618  2.389446  
1604  2.329